Author
======

Ali Yasin Akalin

20492

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

## Custom Functions

In [ ]:
def init():
    from pathlib import Path
    import os

    # DIR_SRC points to project directory which is CS210_Project_1/src
    # and can be reached from definitions.DIR_SRC after
    # running definitions.init() just once
    
    global DIR_SRC
    #DIR_SRC = Path(__file__).parent
    DIR_SRC = Path.cwd().parent / "src"

    global DIR_DATA
    #DIR_DATA = DIR_SRC / 'data'
    DIR_DATA = DIR_SRC / 'data'

    # Changes active directory to project directory
    #os.chdir(DIR_SRC)

### Basic Functions
Open file, save file as csv and print reports.

In [ ]:
def openDataFile(file_name):
    init()
    data_frame = pd.read_csv(DIR_DATA / file_name)
    return data_frame


def saveDataFrame(data_frame,
                  file_name,
                  verbose=False):
    init()
    data_frame.to_csv(DIR_DATA / file_name)
    if verbose:
        print("File saved into", str(DIR_DATA))


def printInfo(info, description='', info_name=''):
    print("============================================")
    if description:
        print(description)
    print(info)

In [ ]:
def showInfo(data_frame):
    printInfo(data_frame.shape, "Shape of the data (rows, columns)")
    printInfo(data_frame.head(), "First 5 rows of the data)")
    printInfo(data_frame.describe(), "Statistical analysis")
    printInfo(data_frame.dtypes, "Data type of columns")


def addColumn(data_frame, column_name, default_value=np.nan):
    data_frame[column_name] = default_value


def dealNanValues(data_frame):
    printInfo(data_frame.isnull().sum())

In [ ]:
# Seaborn parameters
sns.set(style="darkgrid")

# Matplotlib settings
%matplotlib inline

## Execution

In [ ]:
init()
df = openDataFile("taxi-trips.csv")

showInfo(df)

''' Save Updated Dataframe
file_name_2 = "taxi-trips_2.csv"
saveDataFrame(df, file_name_2)
'''